In [3]:
import seq2seq_LSTM
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Specify the number of loops And Locate the bassline and drum dataset files

In [4]:
nbars = 2 

bassline_dataset_file = "../data/bassline_size_50[WithOffet]_translated_to_midi_36-"+str(int(nbars))+"bars.txt"
drum_dataset_file = "../data/drum_size_50.txt"

## Model Architecture (Please understand the code before modifying)

In [6]:
drum_dataset_0b, drum_dataset_int, bassline_dataset = seq2seq_LSTM.get_sequences(bassline_dataset_file, drum_dataset_file)

output_cardinality = len(np.unique(bassline_dataset))+2 # <<<<<<<<<<<<<<<<<<<< +1 is for the starting character
input_cardinality = 2**8 # <<<<<<<<<<<<<<<<<<<<

print("Number of Bassline Unique Words: \n\t\t", output_cardinality)

print("Number of Drum Unique Words: \n\t\t", input_cardinality)
print("Bassline Unique Words:  \n\t\t ", np.unique(bassline_dataset))

# configure problem
n_features_drums = input_cardinality           # 2^8 different unique drum combinations #Cardinality of Input
n_features_bassline = output_cardinality        #Cardinality of Output
n_units = 128

# define model
train, infenc, infdec = seq2seq_LSTM.define_models(input_cardinality, output_cardinality, n_units)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

encoder_input, decoder_input, decoder_output, drum_dataset_int_words_tokens, target_words_tokens, target_in_words_tokens = seq2seq_LSTM.get_dataset(drum_dataset_int, 
                                                                                                                                                    bassline_dataset, 
                                                                                                                                                    padding_value = "S", 
                                                                                                                                                    input_cardinality=input_cardinality, 
                                                                                                                                                    output_cardinality=output_cardinality)

print ("drum_dataset_int_words_tokens \n", drum_dataset_int_words_tokens)
print ("target_words_tokens \n", target_words_tokens)
print ("target_in_words_tokens \n", target_in_words_tokens)

# Show Summary
train.summary()

# plot_model(train, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
plot_model(train, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
SVG(model_to_dot(train).create(prog='dot', format='svg'))



Number of Bassline Unique Words: 
		 36
Number of Drum Unique Words: 
		 256
Bassline Unique Words:  
		  [   0   21   22   23   24   25   26   27   28   29   30   31   32   33
   34   35   36   37   38   39   40   41   42   43   44   45   46   47
   48   49   50   51   54 1000]
drum_dataset_int_words_tokens 
 {0: 1, 1: 2, 3: 3, 5: 4, 6: 5, 7: 6, 8: 7, 135: 8, 15: 9, 143: 10, 16: 11, 20: 12, 23: 13, 24: 14, 31: 15, 160: 16, 32: 17, 159: 18, 39: 19, 47: 20, 56: 21, 63: 22, 192: 23, 199: 24, 71: 25, 200: 26, 207: 27, 88: 28, 216: 29, 95: 30, 224: 31, 223: 32, 96: 33, 228: 34, 231: 35, 232: 36, 234: 37, 127: 38, 239: 39, 240: 40, 247: 41, 248: 42, 249: 43, 251: 44, 252: 45, 253: 46, 255: 47}
target_words_tokens 
 {0: 1, 21: 2, 22: 3, 23: 4, 24: 5, 25: 6, 26: 7, 27: 8, 28: 9, 29: 10, 30: 11, 31: 12, 32: 13, 33: 14, 34: 15, 35: 16, 36: 17, 37: 18, 38: 19, 39: 20, 40: 21, 41: 22, 42: 23, 43: 24, 44: 25, 45: 26, 46: 27, 47: 28, 48: 29, 49: 30, 50: 31, 51: 32, 54: 33, 1000: 34}
target_in_words

AttributeError: 'NoneType' object has no attribute 'create'

## Specify the number of Epochs used for training

In [ ]:
epochs = 4000

## Train the model

In [ ]:
history = train.fit([encoder_input, decoder_input], decoder_output, epochs=epochs)

## Save the Encoder/Decoder models

In [ ]:
infenc.save("infenc_epoch_"+str(epochs)+"_InputCar_"+str(input_cardinality)+
            "_OutputCar_"+str(output_cardinality)+"_n_units_"+str(n_units)+".h5")
infdec.save("infdec_epoch_"+str(epochs)+"_InputCar_"+str(input_cardinality)+
            "_OutputCar_"+str(output_cardinality)+"_n_units_"+str(n_units)+".h5")

## Performance Plots

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 6)

# summarize history for accuracy
plt.subplot(2, 1, 1)
plt.plot(history.history['acc'])
#plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
plt.legend(['train'], loc='upper left')
plt.show()

# summarize history for loss
plt.subplot(2, 1, 2)
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
plt.legend(['train'], loc='upper left')
plt.show()